In [1]:
import numpy as np
import xarray as xr
import os
import fnmatch
from tqdm.autonotebook import tqdm
import dask
import random
import sys
sys.path.insert(0, '/home/jovyan/CMIP6cf/cmip6cf/')

from dependence_metrics import kendallstau, utdc_at_threshold, utdc_cfg

/tmp/ipykernel_1462/3902141566.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def pot(da,threshold,dim):
    
    assert (threshold>=0) & (threshold<1)
    
    return da.where(da>da.quantile(threshold,dim=dim))

def declustered_peaks(da,threshold,window_len,dim):
    #computes peaks above threshold of xr.DataArray and declusters them with a rolling window.
    
    peaks = pot(da,threshold,dim)
    
    return peaks.where(peaks==peaks.rolling({dim:window_len},center=True,min_periods=1).max(skipna=True))

def rolling_max(da,window_len,dim):
    return da.rolling({dim:window_len},center=True,min_periods=1).max()

Configure the bivariate sampling:

In [3]:
max_lag = 0 #days
declus_window_len = 1 #days
threshold = .98 #quantile

output_yrs = np.arange(1880,2100,20)
window_len=40 #may need to increase? indicate settings in output folder?

season = 'year' 
overwrite_output = True

In [4]:
sfcWind_dir = '/home/jovyan/CMIP6cf/output/timeseries/sfcWind_tgs'
pr_dir = '/home/jovyan/CMIP6cf/output/timeseries/pr_tgs'

source_ids = list(set(os.listdir(sfcWind_dir)) & set(os.listdir(pr_dir))) #intersection of models

for source_id in [k for k in source_ids if ~k.startswith('.')]: #loop over models
  
    sfcWind_path = os.path.join(sfcWind_dir,source_id)
    pr_path = os.path.join(pr_dir,source_id)
    
    sfcWind_exps = [s.split('_')[-1][0:-3] for s in os.listdir(sfcWind_path) if s.startswith('.')==False]
    pr_exps = [s.split('_')[-1][0:-3] for s in os.listdir(pr_path) if s.startswith('.')==False]
    
    experiment_ids = list(set(sfcWind_exps) & set(pr_exps))

    for experiment_id in experiment_ids: #loop over experiments
        #load data
        fn = fnmatch.filter(os.listdir(sfcWind_path),'*'+experiment_id+'*')[0]
        print('Processing file: '+fn)
    
        sfcWind_pr = xr.open_mfdataset((os.path.join(sfcWind_path,fn),os.path.join(pr_path,fn)),chunks={'member_id':1,'time':100000,'tg':109})#.sel(longitude=np.arange(-25,11))
        
        #generate output paths & check if output already exists
        #model_path = os.path.join('/home/jovyan/CMIP6cf/output/dependence/sfcWind_pr_europe/40yr_p98_lag0d_declus1d_ref2000',sfcWind_pr.source_id)
        model_path = os.path.join('/home/jovyan/CMIP6cf/output/dependence/sfcWind_pr_tgs/40yr_p98_lag0d_declus1d_ref2000',sfcWind_pr.source_id)
        output_fn = os.path.join(model_path,fn)

        if not os.path.exists(model_path):
            os.mkdir(model_path)

        if not overwrite_output: #if not overwriting existing output
            if os.path.exists(output_fn):
                print('Output already exists for this instance.')
                continue
        
        #construct time window indices
        if len(np.unique(sfcWind_pr.time.resample(time='1Y').count()))>1: #remove leap days so that each computation window has the same length
            with dask.config.set(**{'array.slicing.split_large_chunks': True}):
                sfcWind_pr = sfcWind_pr.sel(time=~((sfcWind_pr.time.dt.month == 2) & (sfcWind_pr.time.dt.day == 29))) #^probably (hopefully) only has a small effect on the results
        
        #select DJF:
        days_in_year = int(sfcWind_pr.time.resample(time='1Y').count()[0])
        
        if window_len%2 !=0: #odd
            window_start_idx = days_in_year*(output_yrs-1850-int(np.floor(window_len/2)))
            first_window_idx = np.arange(0*days_in_year,window_len*days_in_year)
        else: #even
            window_start_idx = days_in_year*(output_yrs-1850-int(window_len/2)+1)
            first_window_idx = np.arange(0*days_in_year,window_len*days_in_year)
        
        if np.max(first_window_idx[:,np.newaxis]+window_start_idx[np.newaxis,:])>=len(sfcWind_pr.time):
            raise Exception('Windows exceed simulation length.')
            
        window_idx = xr.DataArray( #indices of windows
            data=first_window_idx[:,np.newaxis]+window_start_idx[np.newaxis,:],
            dims=["time_in_window_idx","window"],
            coords=dict(
                time_in_window_idx=first_window_idx,
                window=output_yrs
            ),
        )
        
        
        for m,member in tqdm(enumerate(sfcWind_pr.member_id)): #loop over members to compute the dependence
        
            sfcWind_pr_mem = sfcWind_pr.sel(member_id=member)#.copy(deep=True)
            
            sfcWind_pr_wdws = sfcWind_pr_mem.isel(time=window_idx) #select windows
   
            data_is_complete = np.isfinite(sfcWind_pr_wdws.sfcWind).all(dim='time_in_window_idx') * np.isfinite(sfcWind_pr_wdws.pr).all(dim='time_in_window_idx')
        
            pr_hist_threshold = sfcWind_pr_wdws['pr'].sel(window=2000).quantile(threshold,dim='time_in_window_idx')
            sfcWind_hist_threshold = sfcWind_pr_wdws['sfcWind'].sel(window=2000).quantile(threshold,dim='time_in_window_idx')

            pr_peaks = sfcWind_pr_wdws['pr'].where(sfcWind_pr_wdws['pr']>pr_hist_threshold)
            sfcWind_peaks = sfcWind_pr_wdws['sfcWind'].where(sfcWind_pr_wdws['sfcWind']>sfcWind_hist_threshold)
            
            pr_peaks_declustered = pr_peaks.where(pr_peaks==pr_peaks.rolling({'time_in_window_idx':declus_window_len},center=True,min_periods=1).max(skipna=True))
            sfcWind_peaks_declustered = sfcWind_peaks.where(sfcWind_peaks==sfcWind_peaks.rolling({'time_in_window_idx':declus_window_len},center=True,min_periods=1).max(skipna=True))
            
            #decomposition (only works for declustering = 0 and lag =1)?
            
            #sort values in historical period
            #europe:
            sorted_pr_2000 = xr.DataArray(data=np.sort(sfcWind_pr_wdws.sel(window=2000).pr,axis=0),dims=['time_in_window_idx','tg'],
                                          coords=dict(time_in_window_idx=sfcWind_pr_wdws.time_in_window_idx,tg=sfcWind_pr_wdws.tg))
            sorted_sfcWind_2000 = xr.DataArray(data=np.sort(sfcWind_pr_wdws.sel(window=2000).sfcWind,axis=0),dims=['time_in_window_idx','tg'],
                                               coords=dict(time_in_window_idx=sfcWind_pr_wdws.time_in_window_idx,tg=sfcWind_pr_wdws.tg))

            #tgs:
            
            
            #derive percentile values based on number of exceedences of historical 98% in the future & determine peaks, #potentially expand this to other windows than just 2080
            pr_eqv_thresholds = sorted_pr_2000.isel(time_in_window_idx=-1*(np.isfinite(pr_peaks_declustered).sum(dim='time_in_window_idx').sel(window=2080).load()))
            wind_eqv_thresholds = sorted_sfcWind_2000.isel(time_in_window_idx=-1*(np.isfinite(sfcWind_peaks_declustered).sum(dim='time_in_window_idx').sel(window=2080).load()))
            
            pr_peaks_fut_threshold = sfcWind_pr_wdws.pr.sel(window=2000).where(sfcWind_pr_wdws.pr.sel(window=2000)>=pr_eqv_thresholds)
            sfcWind_peaks_fut_threshold = sfcWind_pr_wdws.sfcWind.sel(window=2000).where(sfcWind_pr_wdws.sfcWind.sel(window=2000)>=wind_eqv_thresholds)

            #compute number of co_occurring extremes due to different effects

            #co occurring pr fut threshold
            num_co_occurring_pr_driven = np.isfinite((rolling_max(pr_peaks_fut_threshold,max_lag*2+1,dim='time_in_window_idx')*sfcWind_peaks_declustered.sel(window=2000))).sum(dim='time_in_window_idx')

            #co occurring wind fut threshold 
            num_co_occurring_wind_driven = np.isfinite((rolling_max(pr_peaks_declustered.sel(window=2000),max_lag*2+1,dim='time_in_window_idx')*sfcWind_peaks_fut_threshold)).sum(dim='time_in_window_idx')

            #co occurring both new threshold
            num_co_occurring_both_driven = np.isfinite((rolling_max(pr_peaks_fut_threshold,max_lag*2+1,dim='time_in_window_idx')*sfcWind_peaks_fut_threshold)).sum(dim='time_in_window_idx')
           
            
            #pr_peaks_declustered = declustered_peaks(sfcWind_pr_wdws['pr'],threshold,declus_window_len,dim='time_in_window_idx')
            #sfcWind_peaks_declustered = declustered_peaks(sfcWind_pr_wdws['sfcWind'],threshold,declus_window_len,dim='time_in_window_idx')
            
            #count occurrences of peaks
            co_occurring = np.isfinite((rolling_max(pr_peaks_declustered,max_lag*2+1,dim='time_in_window_idx')*sfcWind_peaks_declustered))
            
            for month in np.arange(1,13):
                if month==1:
                    num_co_occurring_pmonth = co_occurring.where(co_occurring.time.dt.month==month).sum(dim='time_in_window_idx')
                    num_pr_peaks_pmonth = np.isfinite(pr_peaks_declustered.where(pr_peaks_declustered.time.dt.month==month)).sum(dim='time_in_window_idx')
                    num_sfcWind_peaks_pmonth = np.isfinite(sfcWind_peaks_declustered.where(sfcWind_peaks_declustered.time.dt.month==month)).sum(dim='time_in_window_idx')
                else:
                    num_co_occurring_pmonth = xr.concat((num_co_occurring_pmonth,co_occurring.where(co_occurring.time.dt.month==month).sum(dim='time_in_window_idx')),dim='month')
                    num_pr_peaks_pmonth = xr.concat((num_pr_peaks_pmonth,np.isfinite(pr_peaks_declustered.where(pr_peaks_declustered.time.dt.month==month)).sum(dim='time_in_window_idx')),dim='month')
                    num_sfcWind_peaks_pmonth = xr.concat((num_sfcWind_peaks_pmonth,np.isfinite(sfcWind_peaks_declustered.where(sfcWind_peaks_declustered.time.dt.month==month)).sum(dim='time_in_window_idx')),dim='month')
                    
            num_co_occurring_pmonth = num_co_occurring_pmonth.assign_coords({'month':np.arange(1,13)})                             
            num_pr_peaks_pmonth = num_pr_peaks_pmonth.assign_coords({'month':np.arange(1,13)})       
            num_sfcWind_peaks_pmonth = num_sfcWind_peaks_pmonth.assign_coords({'month':np.arange(1,13)})       
            
            #dependence_mem = xr.concat([ktau_pr_cdon_sfcWind,ktau_sfcWind_cdon_pr,ktau_both_peaks],dim='extreme_variate').to_dataset(name='ktau')
                
            dependence_mem = num_co_occurring_pmonth.to_dataset(name='num_co_occurring')
            dependence_mem['num_pr_peaks'] = num_pr_peaks_pmonth
            dependence_mem['num_sfcWind_peaks'] = num_sfcWind_peaks_pmonth
           
            dependence_mem['num_co_occurring_pr_driven']   = num_co_occurring_pr_driven
            dependence_mem['num_co_occurring_sfcWind_driven'] = num_co_occurring_wind_driven
            dependence_mem['num_co_occurring_both_driven'] = num_co_occurring_both_driven
            
            dependence_mem['num_co_occurring_pr_driven'].attrs['window']='2061-2100'
            dependence_mem['num_co_occurring_sfcWind_driven'].attrs['window']='2061-2100'
            dependence_mem['num_co_occurring_both_driven'].attrs['window']='2061-2100'
           
            #store thresholds
            dependence_mem['sfcWind_thresholds_annual'] = sfcWind_pr_wdws['sfcWind'].quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['pr_thresholds_annual'] = sfcWind_pr_wdws['pr'].quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            
            dependence_mem['sfcWind_thresholds_djf'] = sfcWind_pr_wdws['sfcWind'].where(np.mod(sfcWind_pr_wdws.time.dt.month,12)<3).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['pr_thresholds_djf'] = sfcWind_pr_wdws['pr'].where(np.mod(sfcWind_pr_wdws.time.dt.month,12)<3).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            
            dependence_mem['sfcWind_thresholds_mam'] = sfcWind_pr_wdws['sfcWind'].where((np.mod(sfcWind_pr_wdws.time.dt.month,12)>2) & (np.mod(sfcWind_pr_wdws.time.dt.month,12)<6)).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['pr_thresholds_mam'] = sfcWind_pr_wdws['pr'].where((np.mod(sfcWind_pr_wdws.time.dt.month,12)>2) & (np.mod(sfcWind_pr_wdws.time.dt.month,12)<6)).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            
            dependence_mem['sfcWind_thresholds_jja'] = sfcWind_pr_wdws['sfcWind'].where((np.mod(sfcWind_pr_wdws.time.dt.month,12)>5) & (np.mod(sfcWind_pr_wdws.time.dt.month,12)<9)).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['pr_thresholds_jja'] = sfcWind_pr_wdws['pr'].where((np.mod(sfcWind_pr_wdws.time.dt.month,12)>5) & (np.mod(sfcWind_pr_wdws.time.dt.month,12)<9)).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            
            dependence_mem['sfcWind_thresholds_son'] = sfcWind_pr_wdws['sfcWind'].where((np.mod(sfcWind_pr_wdws.time.dt.month,12)>8) & (np.mod(sfcWind_pr_wdws.time.dt.month,12)<12)).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['pr_thresholds_son'] = sfcWind_pr_wdws['pr'].where((np.mod(sfcWind_pr_wdws.time.dt.month,12)>8) & (np.mod(sfcWind_pr_wdws.time.dt.month,12)<12)).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['complete_window'] = data_is_complete #store where windows miss data
            
            
            dependence_mem = dependence_mem.expand_dims(dim={"member_id": 1}) #add coordinates & dimensions
            #dependence_mem = dependence_mem.assign_coords({'extreme_variate':['sfcWind','pr','both'],'statistic':['coef','p'],'estimator':['cfg','.95']})
            '''
            if m==0: #concatenate results over member_id's
                dependence = dependence_mem
            else:
                dependence = xr.concat((dependence,dependence_mem),dim='member_id')    
            '''
            dependence_mem.attrs = sfcWind_pr.attrs #keep original attributes and add information on the extremes analysis
            dependence_mem.attrs['window_length'] = str(window_len)
            dependence_mem.attrs['declustering'] = 'Rolling window of '+str(declus_window_len)+' days'
            dependence_mem.attrs['allowed_lag'] = str(max_lag)

            dependence_mem.to_netcdf(output_fn[0:-3]+'_'+dependence_mem.member_id.values[0]+'.nc',mode='w')
            dependence_mem.close()
        '''    
        dependence.attrs = sfcWind_pr.attrs #keep original attributes and add information on the extremes analysis
        dependence.attrs['window_length'] = str(window_len)
        dependence.attrs['declustering'] = 'Rolling window of '+str(declus_window_len)+' days'
        dependence.attrs['allowed_lag'] = str(max_lag)

        dependence.to_netcdf(output_fn,mode='w')
        dependence.close()
        '''
        #store for all members of this model & experiment


Processing file: CESM2-WACCM_gn_day_ssp585.nc


0it [00:00, ?it/s]

Processing file: CESM2-WACCM_gn_day_ssp245.nc


0it [00:00, ?it/s]

Processing file: CMCC-CM2-SR5_gn_day_ssp585.nc


...

0it [00:00, ?it/s]

Processing file: CMCC-ESM2_gn_day_ssp585.nc


0it [00:00, ?it/s]

Processing file: CMCC-ESM2_gn_day_ssp245.nc


0it [00:00, ?it/s]

```
sfcWind_dir = '/home/jovyan/CMIP6cf/output/timeseries/sfcWind_europe/'
pr_dir = '/home/jovyan/CMIP6cf/output/timeseries/pr_europe/'

source_ids = list(set(os.listdir(sfcWind_dir)) & set(os.listdir(pr_dir))) #intersection of models

for source_id in ['EC-Earth3']:#[k for k in source_ids if ~k.startswith('.')]: #loop over models
  
    sfcWind_path = os.path.join(sfcWind_dir,source_id)
    pr_path = os.path.join(pr_dir,source_id)
    
    sfcWind_exps = [s.split('_')[-1][0:-3] for s in os.listdir(sfcWind_path) if s.startswith('.')==False]
    pr_exps = [s.split('_')[-1][0:-3] for s in os.listdir(pr_path) if s.startswith('.')==False]
    
    experiment_ids = list(set(sfcWind_exps) & set(pr_exps))

    for experiment_id in experiment_ids: #loop over experiments
        #load data
        fn = fnmatch.filter(os.listdir(sfcWind_path),'*'+experiment_id+'*')[0]
        print('Processing file: '+fn)
        sfcWind_pr = xr.open_mfdataset((os.path.join(sfcWind_path,fn),os.path.join(pr_path,fn)),chunks={'member_id':1,'time':100000,'longitude':5})#.sel(longitude=np.arange(-25,11))
        
        #generate output paths & check if output already exists
        model_path = os.path.join('/home/jovyan/CMIP6cf/output/dependence/sfcWind_pr_europe/40yr_p98_lag0d_declus1d_ref1980',sfcWind_pr.source_id)
        output_fn = os.path.join(model_path,fn)

        if not os.path.exists(model_path):
            os.mkdir(model_path)

        if not overwrite_output: #if not overwriting existing output
            if os.path.exists(output_fn):
                print('Output already exists for this instance.')
                continue
        
        #construct time window indices
        if len(np.unique(sfcWind_pr.time.resample(time='1Y').count()))>1: #remove leap days so that each computation window has the same length
            with dask.config.set(**{'array.slicing.split_large_chunks': True}):
                sfcWind_pr = sfcWind_pr.sel(time=~((sfcWind_pr.time.dt.month == 2) & (sfcWind_pr.time.dt.day == 29))) #^probably (hopefully) only has a small effect on the results
        
        #select DJF:
        days_in_year = int(sfcWind_pr.time.resample(time='1Y').count()[0])
        
        if window_len%2 !=0: #odd
            window_start_idx = days_in_year*(output_yrs-1850-int(np.floor(window_len/2)))
            first_window_idx = np.arange(0*days_in_year,window_len*days_in_year)
        else: #even
            window_start_idx = days_in_year*(output_yrs-1850-int(window_len/2)+1)
            first_window_idx = np.arange(0*days_in_year,window_len*days_in_year)
        
        if np.max(first_window_idx[:,np.newaxis]+window_start_idx[np.newaxis,:])>=len(sfcWind_pr.time):
            raise Exception('Windows exceed simulation length.')
            
        window_idx = xr.DataArray( #indices of windows
            data=first_window_idx[:,np.newaxis]+window_start_idx[np.newaxis,:],
            dims=["time_in_window_idx","window"],
            coords=dict(
                time_in_window_idx=first_window_idx,
                window=output_yrs
            ),
        )
        
        for m,member in tqdm(enumerate(sfcWind_pr.member_id)): #loop over members to compute the dependence
            sfcWind_pr_mem = sfcWind_pr.sel(member_id=member).copy(deep=True).load()
            
            sfcWind_pr_wdws = sfcWind_pr_mem.isel(time=window_idx) #select windows
   
            data_is_complete = np.isfinite(sfcWind_pr_wdws.sfcWind).all(dim='time_in_window_idx') * np.isfinite(sfcWind_pr_wdws.pr).all(dim='time_in_window_idx')
            
            pr_hist_threshold = sfcWind_pr_wdws['pr'].sel(window=1980).quantile(threshold,dim='time_in_window_idx')
            sfcWind_hist_threshold = sfcWind_pr_wdws['sfcWind'].sel(window=1980).quantile(threshold,dim='time_in_window_idx')

            pr_peaks = sfcWind_pr_wdws['pr'].where(sfcWind_pr_wdws['pr']>pr_hist_threshold)
            sfcWind_peaks = sfcWind_pr_wdws['sfcWind'].where(sfcWind_pr_wdws['sfcWind']>sfcWind_hist_threshold)
            
            pr_peaks_declustered = pr_peaks.where(pr_peaks==pr_peaks.rolling({'time_in_window_idx':declus_window_len},center=True,min_periods=1).max(skipna=True))
            sfcWind_peaks_declustered = sfcWind_peaks.where(sfcWind_peaks==sfcWind_peaks.rolling({'time_in_window_idx':declus_window_len},center=True,min_periods=1).max(skipna=True))
            
            #decomposition (only works for declustering = 0 and lag =1)?
            
            #sort values in historical period
         
            sorted_pr_1980 = xr.DataArray(data=np.sort(sfcWind_pr_wdws.sel(window=1980).pr,axis=0),dims=['time_in_window_idx','latitude','longitude'],
                                          coords=dict(time_in_window_idx=sfcWind_pr_wdws.time_in_window_idx,latitude=sfcWind_pr_wdws.latitude,longitude=sfcWind_pr_wdws.longitude)).chunk({'longitude':5})
            sorted_sfcWind_1980 = xr.DataArray(data=np.sort(sfcWind_pr_wdws.sel(window=1980).sfcWind,axis=0),dims=['time_in_window_idx','latitude','longitude'],
                                               coords=dict(time_in_window_idx=sfcWind_pr_wdws.time_in_window_idx,latitude=sfcWind_pr_wdws.latitude,longitude=sfcWind_pr_wdws.longitude)).chunk({'longitude':5})

            #derive percentile values based on number of exceedences of historical 98% in the future & determine peaks, #potentially expand this to other windows than just 2080
            pr_eqv_thresholds = sorted_pr_1980.isel(time_in_window_idx=-1*(np.isfinite(pr_peaks_declustered).sum(dim='time_in_window_idx').sel(window=2080).load()))
            wind_eqv_thresholds = sorted_sfcWind_1980.isel(time_in_window_idx=-1*(np.isfinite(sfcWind_peaks_declustered).sum(dim='time_in_window_idx').sel(window=2080).load()))
            
            pr_peaks_fut_threshold = sfcWind_pr_wdws.pr.sel(window=1980).where(sfcWind_pr_wdws.pr.sel(window=1980)>=pr_eqv_thresholds)
            sfcWind_peaks_fut_threshold = sfcWind_pr_wdws.sfcWind.sel(window=1980).where(sfcWind_pr_wdws.sfcWind.sel(window=1980)>=wind_eqv_thresholds)

            #compute number of co_occurring extremes due to different effects

            #co occurring pr fut threshold
            num_co_occurring_pr_driven = np.isfinite((rolling_max(pr_peaks_fut_threshold,max_lag*2+1,dim='time_in_window_idx')*sfcWind_peaks_declustered.sel(window=1980))).sum(dim='time_in_window_idx')

            #co occurring wind fut threshold 
            num_co_occurring_wind_driven = np.isfinite((rolling_max(pr_peaks_declustered.sel(window=1980),max_lag*2+1,dim='time_in_window_idx')*sfcWind_peaks_fut_threshold)).sum(dim='time_in_window_idx')

            #co occurring both new threshold
            num_co_occurring_both_driven = np.isfinite((rolling_max(pr_peaks_fut_threshold,max_lag*2+1,dim='time_in_window_idx')*sfcWind_peaks_fut_threshold)).sum(dim='time_in_window_idx')
           
            
            #pr_peaks_declustered = declustered_peaks(sfcWind_pr_wdws['pr'],threshold,declus_window_len,dim='time_in_window_idx')
            #sfcWind_peaks_declustered = declustered_peaks(sfcWind_pr_wdws['sfcWind'],threshold,declus_window_len,dim='time_in_window_idx')
            
            #count occurrences of peaks
            co_occurring = np.isfinite((rolling_max(pr_peaks_declustered,max_lag*2+1,dim='time_in_window_idx')*sfcWind_peaks_declustered))
            
            for month in np.arange(1,13):
                if month==1:
                    num_co_occurring_pmonth = co_occurring.where(co_occurring.time.dt.month==month).sum(dim='time_in_window_idx')
                    num_pr_peaks_pmonth = np.isfinite(pr_peaks_declustered.where(pr_peaks_declustered.time.dt.month==month)).sum(dim='time_in_window_idx')
                    num_sfcWind_peaks_pmonth = np.isfinite(sfcWind_peaks_declustered.where(sfcWind_peaks_declustered.time.dt.month==month)).sum(dim='time_in_window_idx')
                else:
                    num_co_occurring_pmonth = xr.concat((num_co_occurring_pmonth,co_occurring.where(co_occurring.time.dt.month==month).sum(dim='time_in_window_idx')),dim='month')
                    num_pr_peaks_pmonth = xr.concat((num_pr_peaks_pmonth,np.isfinite(pr_peaks_declustered.where(pr_peaks_declustered.time.dt.month==month)).sum(dim='time_in_window_idx')),dim='month')
                    num_sfcWind_peaks_pmonth = xr.concat((num_sfcWind_peaks_pmonth,np.isfinite(sfcWind_peaks_declustered.where(sfcWind_peaks_declustered.time.dt.month==month)).sum(dim='time_in_window_idx')),dim='month')
                    
            num_co_occurring_pmonth = num_co_occurring_pmonth.assign_coords({'month':np.arange(1,13)})                             
            num_pr_peaks_pmonth = num_pr_peaks_pmonth.assign_coords({'month':np.arange(1,13)})       
            num_sfcWind_peaks_pmonth = num_sfcWind_peaks_pmonth.assign_coords({'month':np.arange(1,13)})       
            
            #dependence_mem = xr.concat([ktau_pr_cdon_sfcWind,ktau_sfcWind_cdon_pr,ktau_both_peaks],dim='extreme_variate').to_dataset(name='ktau')
                
            dependence_mem = num_co_occurring_pmonth.to_dataset(name='num_co_occurring')
            dependence_mem['num_pr_peaks'] = num_pr_peaks_pmonth
            dependence_mem['num_sfcWind_peaks'] = num_sfcWind_peaks_pmonth
           
            dependence_mem['num_co_occurring_pr_driven']   = num_co_occurring_pr_driven
            dependence_mem['num_co_occurring_sfcWind_driven'] = num_co_occurring_wind_driven
            dependence_mem['num_co_occurring_both_driven'] = num_co_occurring_both_driven
            
            dependence_mem['num_co_occurring_pr_driven'].attrs['window']='2061-2100'
            dependence_mem['num_co_occurring_sfcWind_driven'].attrs['window']='2061-2100'
            dependence_mem['num_co_occurring_both_driven'].attrs['window']='2061-2100'
           
            #store thresholds
            dependence_mem['sfcWind_thresholds_annual'] = sfcWind_pr_wdws['sfcWind'].quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['pr_thresholds_annual'] = sfcWind_pr_wdws['pr'].quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            
            dependence_mem['sfcWind_thresholds_djf'] = sfcWind_pr_wdws['sfcWind'].where(np.mod(sfcWind_pr_wdws.time.dt.month,12)<3).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['pr_thresholds_djf'] = sfcWind_pr_wdws['pr'].where(np.mod(sfcWind_pr_wdws.time.dt.month,12)<3).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            
            dependence_mem['sfcWind_thresholds_mam'] = sfcWind_pr_wdws['sfcWind'].where((np.mod(sfcWind_pr_wdws.time.dt.month,12)>2) & (np.mod(sfcWind_pr_wdws.time.dt.month,12)<6)).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['pr_thresholds_mam'] = sfcWind_pr_wdws['pr'].where((np.mod(sfcWind_pr_wdws.time.dt.month,12)>2) & (np.mod(sfcWind_pr_wdws.time.dt.month,12)<6)).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            
            dependence_mem['sfcWind_thresholds_jja'] = sfcWind_pr_wdws['sfcWind'].where((np.mod(sfcWind_pr_wdws.time.dt.month,12)>5) & (np.mod(sfcWind_pr_wdws.time.dt.month,12)<9)).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['pr_thresholds_jja'] = sfcWind_pr_wdws['pr'].where((np.mod(sfcWind_pr_wdws.time.dt.month,12)>5) & (np.mod(sfcWind_pr_wdws.time.dt.month,12)<9)).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            
            dependence_mem['sfcWind_thresholds_son'] = sfcWind_pr_wdws['sfcWind'].where((np.mod(sfcWind_pr_wdws.time.dt.month,12)>8) & (np.mod(sfcWind_pr_wdws.time.dt.month,12)<12)).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['pr_thresholds_son'] = sfcWind_pr_wdws['pr'].where((np.mod(sfcWind_pr_wdws.time.dt.month,12)>8) & (np.mod(sfcWind_pr_wdws.time.dt.month,12)<12)).quantile(np.arange(.9,1,.01),dim='time_in_window_idx')
            dependence_mem['complete_window'] = data_is_complete #store where windows miss data
            
            
            dependence_mem = dependence_mem.expand_dims(dim={"member_id": 1}) #add coordinates & dimensions
            #dependence_mem = dependence_mem.assign_coords({'extreme_variate':['sfcWind','pr','both'],'statistic':['coef','p'],'estimator':['cfg','.95']})
            
            if m==0: #concatenate results over member_id's
                dependence = dependence_mem
            else:
                dependence = xr.concat((dependence,dependence_mem),dim='member_id')    
            
            dependence.attrs = sfcWind_pr.attrs #keep original attributes and add information on the extremes analysis
            dependence.attrs['window_length'] = str(window_len)
            dependence.attrs['declustering'] = 'Rolling window of '+str(declus_window_len)+' days'
            dependence.attrs['allowed_lag'] = str(max_lag)
            
            #store for all members of this model & experiment
            dependence.to_netcdf(output_fn,mode='w')
            dependence.close()
            ```